In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
os.environ[
    'XLA_FLAGS'] = '--xla_gpu_cuda_data_dir=/usr/local/cuda'

In [2]:
%pylab inline

import jax
import jax.numpy as jnp
import jax_cosmo as jc
import jaxpm
from jax.experimental.ode import odeint

from jaxpm.painting import cic_paint
from jaxpm.pm import linear_field, lpt, make_ode_fn, make_neural_ode_fn
import pickle
import haiku as hk
from jaxpm.nn import NeuralSplineFourierFilter

%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


/local/home/dl264294/.local/lib/python3.8/site-packages/haiku/_src/data_structures.py:37: FutureWarning: jax.tree_structure is deprecated, and will be removed in a future release. Use jax.tree_util.tree_structure instead.
  PyTreeDef = type(jax.tree_structure(None))


In [3]:
params =pickle.load(open('/local/home/dl264294/flowpm/notebooks/camels_25_64_pkloss.params', 'rb'))

/local/home/dl264294/.local/lib/python3.8/site-packages/haiku/_src/data_structures.py:206: FutureWarning: jax.tree_flatten is deprecated, and will be removed in a future release. Use jax.tree_util.tree_flatten instead.
  leaves, structure = jax.tree_flatten(mapping)


In [4]:
Omega_c= 0.2589
Omega_b=0.04860
sigma8= 0.8159
n_s= 0.9667
h= 0.6774
w0= -1.0
mesh_shape= [128,128,128]
box_size  = [205.,205.,205.]


In [5]:
cosmo = jc.Planck15(Omega_c=Omega_c,Omega_b=Omega_b,sigma8=sigma8,n_s=n_s,h=h,w0=w0)
r=jnp.linspace(0., box_size[0]*11,11+1)
r_center = 0.5 * (r[1:] + r[:-1])
a_center = jc.background.a_of_chi(cosmo, r_center)
stages = a_center[::-1]

/local/home/dl264294/.local/lib/python3.8/site-packages/jax_cosmo/scipy/interpolate.py:35: UserWarning: Explicitly requested dtype <class 'jax.numpy.int64'> requested in astype is not available, and will be truncated to dtype int32. To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment variable. See https://github.com/google/jax#current-gotchas for more.
  s = np.sign(np.clip(x, xp[1], xp[-2]) - xi).astype(np.int64)
/local/home/dl264294/.local/lib/python3.8/site-packages/jax_cosmo/scipy/interpolate.py:36: UserWarning: Explicitly requested dtype <class 'jax.numpy.int64'> requested in astype is not available, and will be truncated to dtype int32. To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment variable. See https://github.com/google/jax#current-gotchas for more.
  a = (fp[ind + np.copysign(1, s).astype(np.int64)] - fp[ind]) / (
/local/home/dl264294/.local/lib/python3.8/site-pac

In [6]:
a0=0.14285714254594556
k = jnp.logspace(-4, 1, 128)
pk = jc.power.linear_matter_power(jc.Planck15(Omega_c=Omega_c,Omega_b=Omega_b,sigma8=sigma8,n_s=n_s,h=h,w0=w0), k)
pk_fn = lambda x: jc.scipy.interpolate.interp(x.reshape([-1]), k, pk).reshape(x.shape)

# Create initial conditions
initial_conditions = linear_field(mesh_shape, box_size, pk_fn, seed=jax.random.PRNGKey(0))

# Create particles
particles = jnp.stack(jnp.meshgrid(*[jnp.arange(s) for s in mesh_shape]),axis=-1).reshape([-1,3])


# Initial displacement
dx, p, f = lpt(cosmo, initial_conditions, particles, 0.1)


In [7]:
@jax.jit
def run_simulation_ode(Omega_c, sigma8, Omega_b, n_s, h,w0):
    res=odeint(make_ode_fn(mesh_shape), [particles+dx, p], jnp.array(stages), cosmo, rtol=1e-5, atol=1e-5)
    return res[0]

In [8]:
%timeit res = run_simulation_ode(Omega_c=0.2589, sigma8=0.8159, Omega_b=0.04860, n_s=0.9667, h=0.6774, w0=-1.0)

2023-07-02 16:06:33.800978: E external/org_tensorflow/tensorflow/compiler/xla/service/slow_operation_alarm.cc:65] Constant folding an instruction is taking > 1s:

  add.28 (displaying the full instruction incurs a runtime overhead. Raise your logging level to 4 or above).

This isn't necessarily a bug; constant-folding is inherently a trade-off between compilation time and speed at runtime.  XLA has some guards that attempt to keep constant folding from taking too long, but fundamentally you'll always be able to come up with an input program that takes a long time.

If you'd like to file a bug, run with envvar XLA_FLAGS=--xla_dump_to=/tmp/foo and attach the results.
2023-07-02 16:06:36.721856: E external/org_tensorflow/tensorflow/compiler/xla/service/slow_operation_alarm.cc:133] The operation took 3.920981573s
Constant folding an instruction is taking > 1s:

  add.28 (displaying the full instruction incurs a runtime overhead. Raise your logging level to 4 or above).

This isn't necessa

182 ms ± 1.54 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [9]:
  model = hk.without_apply_rng(hk.transform(lambda x,a : NeuralSplineFourierFilter(n_knots=16, latent_size=32)(x,a)))

In [10]:
@jax.jit
def run_simulation_neural(Omega_c, sigma8, Omega_b, n_s, h,w0):
    res=odeint(make_neural_ode_fn(model,mesh_shape), [particles+dx, p], jnp.array(stages), cosmo, params, rtol=1e-5, atol=1e-5)
    return res[0]

In [11]:
%timeit tes=run_simulation_neural(Omega_c=0.2589, sigma8=0.8159, Omega_b=0.04860, n_s=0.9667, h=0.6774, w0=-1.0)

/local/home/dl264294/.local/lib/python3.8/site-packages/haiku/_src/data_structures.py:214: FutureWarning: jax.tree_unflatten is deprecated, and will be removed in a future release. Use jax.tree_util.tree_unflatten instead.
  self._mapping = jax.tree_unflatten(self._structure, self._leaves)


242 ms ± 1.51 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
